In [3]:
import numpy as np

In [4]:
a=np.array([1,2,3])
2*a

array([2, 4, 6])

In [5]:
a=np.array([[1,2]])
a.shape

(1, 2)

In [7]:
2*(np.transpose(a))@a

array([[2, 4],
       [4, 8]])

In [51]:
def add_col(A,m,i,j):
    n=A.shape[1]
    E=np.eye(n)
    if i==j:
        E[i,i]=m+1
    else:
        E[j,i]=m
    return A@E
def det1(A):
    ZERO=1.0e-100
    n=A.shape[0]
    B=A
    for i in range(n):
        for j in range(n):
            B[i,j]=float(B[i,j])
    d=1
    for i in range(n):
        if abs(B[i,i])<=ZERO:
            x=0
            for col in range(i+1,n):
                if B[i,col]!=0:
                    B=add_col(B,1,i,col)
                    break
                else:
                    x+=1
            if x==n-1-i:
                d=0
                break
            else:
                for row in range(i+1,n):
                    for col in range(n-1,i-1,-1):
                        B[row,col]-=B[row,i]*B[i,col]/B[i,i]
                d=d*B[i,i]
        else:
            for row in range(i+1,n):
                for col in range(n-1,i-1,-1):
                    B[row,col]-=B[row,i]*B[i,col]/B[i,i]
            d=d*B[i,i]
#     d=round(d,5)
    return d

In [52]:
def inv(A):
    '''
    Returns the inverse of a NON-SINGULAR SQUARE matrix
    '''
    n=A.shape[0]
    ZERO=1.0e-100
    d=det1(A)
    if abs(d)<=ZERO:
        s="Inverse is NOT possible:This is a singular matrix"
        return s
    A_inv=np.zeros(n*n)
    A_inv=A_inv.reshape(n,n)
    for i in range(n):
        for j in range(n):
            B=np.array([])
            for k in range((n-1)**2):
                B=np.append(B,0.0)
            B=B.reshape(n-1,n-1)
            for row in range(i):
                for col in range(j):
                    B[row,col]=A[row,col]
                for col in range(j+1,n):
                    B[row,col-1]=A[row,col]
            for row in range(i+1,n):
                for col in range(0,j):
                    B[row-1,col]=A[row,col]
                for col in range(j+1,n):
                    B[row-1,col-1]=A[row,col]
            #Now B has become the minor of A[i,j]
            #So cofactor=((-1)**(i+j))*B
            A_inv[i,j]=((-1)**(i+j))*det1(B)/d
#             A_inv[i,j]=round(A_inv[i,j],5)
    A_inv=A_inv. T
    return A_inv

In [1]:
def dot_val(th,X):
    """
    th - (n+1)x1
    X - 1x(n+1)
    """ 
    import numpy as np
    out=X@th
    return out[0][0]
def hval(th,X):
    """
    th - (n+1)x1
    X - 1x(n+1)
    """ 
    from math import exp
#     print(dot_val(th,X))
    out=1/(1+exp((-1)*float(dot_val(th,X))))
    return out
def probab(th,X,y):
    """
    X - mx(n+1)
    y- mx1
    th - (n+1)x1
    """
    from math import log
    ZERO=1.0e-16
    m=y.shape[0]
    n=th.shape[0]-1
    s=0.0
    for i in range(m):
        t=X[i].reshape(1,n+1)
        ht=hval(th,t)
        temp=y[i][0]
#         if (abs(ht)<=ZERO or abs(1-ht)<=ZERO):
#             return 'c'
#         print(ht)
        s+=temp*log(ht)+(1-temp)*log(1-ht)
    return (-1)*s/m

In [2]:
def repl_th(th,X,y):
    """
    th - (n+1)x1
    X - mx(n+1)
    y- mx1
    """
    import numpy as np
    ZERO=1.0e-16
    m=y.shape[0]
    n=th.shape[0]-1
    H=np.zeros((n+1)**2)
    H=H.reshape(n+1,n+1)
    dJ=np.zeros(n+1)
    dJ=dJ.reshape(n+1,1)
    for i in range(m):
        t1=X[i].reshape(1,n+1)
        t2=np.transpose(t1)
        ht=hval(th,t1)
        H=H+(ht*(1-ht))*(t2@t1)
        dJ=dJ+(ht-y[i][0])*t2
    H=H/m
#     print(H)
#     print(np.linalg.det(H))
#     dJ=np.zeros(n+1)
#     dJ=dJ.reshape(n+1,1)
#     for i in range(m):
#         t1=X[i].reshape(1,n+1)
#         t2=np.transpose(t1)
#         dJ=dJ+(hval(th,t1)-y[i][0])*t2
    dJ=dJ/m
#     print(dJ)
#     if (abs(np.linalg.det(H))<=ZERO):
#         return 'd'
    temp1=np.linalg.pinv(H)
    temp=(temp1)@dJ
#     print(temp1)
#     print(dJ)
#     print(temp)
    for j in range(n+1):
        th[j]-=temp[j]
#     print(th)
    return 0

In [3]:
def newt_meth(X,y):
    """
    X - mx(n+1)
    y- mx1
    test - 1x(n+1)
    """
    ERR=1.0e-10
    import numpy as np
    #alp=1.0e-5
    m=y.shape[0]
    n=X.shape[1]-1
    th=np.zeros(n+1)
    th=th.reshape(n+1,1)
    j0=probab(th,X,y)
    conv=np.Infinity
    print(j0)
    it=0
    while (abs(conv)>=ERR):
        if (repl_th(th,X,y)=='d'):
            break
        j1=probab(th,X,y)
        if (j1=='c'):
            break
        #print(j1)
        conv=j1-j0
        print(conv)
        j0=j1
        it+=1
    print(conv)
    print(it)
    return th
#     if hval(th,test)>=0.5:
#         return(1,it)
#     return(0,it)

In [4]:
us=[]
for i in range(17):
    us.append(i)
del us[11]
us

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16]

In [5]:
import numpy as np
X=np.loadtxt("../Epoch/Selection_Hackathon/Dataset.csv",delimiter=',',skiprows=321,usecols=us)
X=np.c_[np.ones(1279),X]
for i in range(1279):
    X[i][1]/=10
    X[i][6]/=10
    X[i][7]/=10
    X[i][11]/=10
    X[i][12]/=10
    X[i][15]/=100
#     X[i,1:18]/=100
print(X)
print(X.shape)

[[1.         0.98       0.66       ... 0.793      0.28095238 1.0795    ]
 [1.         0.93       0.61       ... 0.71       0.348      1.117     ]
 [1.         0.78       0.62       ... 0.709      0.3        0.7395    ]
 ...
 [1.         0.63       0.51       ... 0.826      0.13793103 0.754     ]
 [1.         0.59       0.645      ... 0.785      0.1375     0.6615    ]
 [1.         0.6        0.31       ... 0.727      0.23333333 1.2075    ]]
(1279, 17)


In [6]:
te=np.loadtxt("../Epoch/Selection_Hackathon/Dataset.csv",delimiter=',',skiprows=1,max_rows=320,usecols=us)
#print(te.shape)
te=np.c_[np.ones(320),te]
for i in range(320):
    te[i][1]/=10
    te[i][6]/=10
    te[i][7]/=10
    te[i][11]/=10
    te[i][12]/=10
    te[i][15]/=100
#     te[i,1:18]/=100
print(te)
print(te.shape)

[[1.         0.74       0.7        ... 0.636      0.30909091 0.608     ]
 [1.         0.78       0.88       ... 0.778      0.268      0.829     ]
 [1.         0.78       0.76       ... 0.742      0.36       0.744     ]
 ...
 [1.         0.96       0.77       ... 0.722      0.24666667 0.935     ]
 [1.         0.98       0.66       ... 0.793      0.28095238 1.0795    ]
 [1.         0.96       0.77       ... 0.722      0.24666667 0.935     ]]
(320, 17)


In [7]:
teans=np.loadtxt("../Epoch/Selection_Hackathon/Dataset.csv",delimiter=',',skiprows=1,max_rows=320,usecols=[11])
teans=teans.reshape(320,1)
print(teans.shape)

(320, 1)


In [8]:
y=np.loadtxt("../Epoch/Selection_Hackathon/Dataset.csv",delimiter=',',skiprows=321,usecols=[11])
y=y.reshape(1279,1)
print(y)

[[1.]
 [0.]
 [0.]
 ...
 [1.]
 [0.]
 [1.]]


In [9]:
th7=newt_meth(X,y)
th7

0.6931471805599231
-0.17648750786897494
-0.012297982529916096
-0.0004325893489899313
-8.84141955204143e-07
-7.34323712947571e-12
-7.34323712947571e-12
5


array([[ 3.31231127e+01],
       [ 2.62897267e-01],
       [-2.70825071e+00],
       [-3.72235801e-01],
       [ 5.77473268e-01],
       [-3.77840347e+00],
       [ 2.23110454e-01],
       [-1.44412066e-01],
       [-3.89538990e+01],
       [-1.29963753e+00],
       [ 3.97708873e+00],
       [ 9.19020308e+00],
       [-7.92798995e-03],
       [-3.96399894e-02],
       [ 1.98685434e-01],
       [ 5.84064085e-01],
       [-1.79040691e+00]])

In [12]:
c=0
for i in range(320):
    temp=te[i].reshape(1,17)
    if (hval(th7,temp)>=0.5299 and teans[i]==1.0):
        c+=1
    elif (hval(th7,temp)<0.5299 and teans[i]==0.0):
        c+=1
c

223

In [107]:
import pandas as pd
df=pd.DataFrame()

In [82]:
th1=newt_meth(X,y)
th1

0.6931471805599231
[[-7.07584050e-02]
 [-6.43080532e-03]
 [-8.09616888e-05]
 [-3.65050821e-04]
 [-1.74569977e-03]
 [-3.98240813e-05]
 [-8.07662236e-03]
 [-5.66849101e-04]
 [-7.03332760e-04]
 [-2.33631744e-03]
 [-6.81821736e-04]
 [-9.86376075e-03]
 [-6.51176701e-03]
 [-3.25588350e-03]
 [-7.21645817e-04]
 [-1.25149354e-02]
 [-6.16632134e-04]]
[[ 6.55814465e+06  1.38797089e+05 -9.49670491e+02 -2.77334260e+08
  -4.15712708e+08 -1.12828423e+10 -5.46992055e+03  1.53685354e+03
  -6.69309930e+08  3.34448196e+06 -1.05895502e+10 -6.44570800e+05
   8.04967012e+09 -1.60992683e+10  1.05901515e+10  1.62718835e+03
   1.38659123e+09]
 [ 7.82530354e+07  1.05770656e+06  1.20383971e+05 -2.16609123e+17
  -3.24913685e+17 -1.80516938e+19 -6.70281158e+04  1.53946502e+04
  -8.00989837e+09  4.76154541e+07 -1.75101709e+19 -8.29814782e+06
   1.36653685e+19 -2.73307369e+19  1.75101709e+19  2.27313988e+04
   1.08304562e+18]
 [ 7.81145401e+07 -1.47595834e+06  1.36362716e+05 -2.16609123e+17
  -3.24913685e+17 -1.8051

OverflowError: math range error

In [63]:
c=0
for i in range(1599):
    temp=te[i].reshape(1,17)
    if (hval(th1,temp)>=0.5 and teans[i]==1.0):
        c+=1
    elif (hval(th1,temp)<0.5 and teans[i]==0.0):
        c+=1
c

1179

In [59]:
te=np.loadtxt("../Epoch/Selection_Hackathon/Dataset.csv",delimiter=',',skiprows=1,usecols=us)
#print(te.shape)
te=np.c_[np.ones(1599),te]
for i in range(1599):
    te[i][1]/=10
    te[i][6]/=10
    te[i][7]/=10
    te[i][11]/=10
    te[i][12]/=10
    te[i][15]/=100
print(te)
print(te.shape)

[[1.         0.74       0.7        ... 0.636      0.30909091 0.608     ]
 [1.         0.78       0.88       ... 0.778      0.268      0.829     ]
 [1.         0.78       0.76       ... 0.742      0.36       0.744     ]
 ...
 [1.         0.63       0.51       ... 0.826      0.13793103 0.754     ]
 [1.         0.59       0.645      ... 0.785      0.1375     0.6615    ]
 [1.         0.6        0.31       ... 0.727      0.23333333 1.2075    ]]
(1599, 17)


In [60]:
teans=np.loadtxt("../Epoch/Selection_Hackathon/Dataset.csv",delimiter=',',skiprows=1,usecols=[11])
teans=teans.reshape(1599,1)
print(teans.shape)

(1599, 1)


In [87]:
import numpy as np
x=np.loadtxt("../Epoch/Selection_Hackathon/Dataset.csv",delimiter=',',skiprows=321)
x=np.c_[np.ones(1279),x]
for i in range(1279):
    x[i][1]/=10
    x[i][6]/=10
    x[i][7]/=10
    x[i][11]/=10
    x[i][13]/=10
    x[i][16]/=100
#     X[i,1:18]/=100
print(x)
print(x.shape)
print(x[0])

[[1.         0.98       0.66       ... 0.793      0.28095238 1.0795    ]
 [1.         0.93       0.61       ... 0.71       0.348      1.117     ]
 [1.         0.78       0.62       ... 0.709      0.3        0.7395    ]
 ...
 [1.         0.63       0.51       ... 0.826      0.13793103 0.754     ]
 [1.         0.59       0.645      ... 0.785      0.1375     0.6615    ]
 [1.         0.6        0.31       ... 0.727      0.23333333 1.2075    ]]
(1279, 18)
[1.         0.98       0.66       0.39       3.2        0.083
 2.1        5.9        0.9989     3.37       0.71       1.15
 1.         1.046      5.23       0.793      0.28095238 1.0795    ]


In [88]:
x=np.unique(x,axis=0)
x

array([[1.        , 0.49      , 0.42      , ..., 0.788     , 0.2625    ,
        0.654     ],
       [1.        , 0.5       , 0.38      , ..., 0.798     , 0.23076923,
        0.506     ],
       [1.        , 0.5       , 0.4       , ..., 0.706     , 0.27586207,
        1.413     ],
       ...,
       [1.        , 1.56      , 0.645     , ..., 0.835     , 0.23      ,
        1.4055    ],
       [1.        , 1.56      , 0.685     , ..., 0.78      , 0.71666667,
        1.312     ],
       [1.        , 1.59      , 0.36      , ..., 0.936     , 0.32272727,
        2.428     ]])

In [89]:
y=x[:,12]
y=y.reshape(1081,1)

In [90]:
x1=np.delete(x,12,axis=1)
x1

array([[1.        , 0.49      , 0.42      , ..., 0.788     , 0.2625    ,
        0.654     ],
       [1.        , 0.5       , 0.38      , ..., 0.798     , 0.23076923,
        0.506     ],
       [1.        , 0.5       , 0.4       , ..., 0.706     , 0.27586207,
        1.413     ],
       ...,
       [1.        , 1.56      , 0.645     , ..., 0.835     , 0.23      ,
        1.4055    ],
       [1.        , 1.56      , 0.685     , ..., 0.78      , 0.71666667,
        1.312     ],
       [1.        , 1.59      , 0.36      , ..., 0.936     , 0.32272727,
        2.428     ]])

In [91]:
te=np.loadtxt("../Epoch/Selection_Hackathon/Dataset.csv",delimiter=',',skiprows=1,max_rows=320,usecols=us)
#print(te.shape)
te=np.c_[np.ones(320),te]
for i in range(320):
    te[i][1]/=10
    te[i][6]/=10
    te[i][7]/=10
    te[i][11]/=10
    te[i][12]/=10
    te[i][15]/=100
#     te[i,1:18]/=100
print(te)
print(te.shape)

[[1.         0.74       0.7        ... 0.636      0.30909091 0.608     ]
 [1.         0.78       0.88       ... 0.778      0.268      0.829     ]
 [1.         0.78       0.76       ... 0.742      0.36       0.744     ]
 ...
 [1.         0.96       0.77       ... 0.722      0.24666667 0.935     ]
 [1.         0.98       0.66       ... 0.793      0.28095238 1.0795    ]
 [1.         0.96       0.77       ... 0.722      0.24666667 0.935     ]]
(320, 17)


In [92]:
teans=np.loadtxt("../Epoch/Selection_Hackathon/Dataset.csv",delimiter=',',skiprows=1,max_rows=320,usecols=[11])
teans=teans.reshape(320,1)
print(teans.shape)

(320, 1)


In [94]:
th10=newt_meth(x1,y)

0.6931471805599291
[[-0.06984274]
 [-0.06347364]
 [-0.00681545]
 [-0.03710916]
 [-0.16829325]
 [-0.00379648]
 [-0.07567068]
 [ 0.00247456]
 [-0.06941538]
 [-0.23111933]
 [-0.06814061]
 [-0.09837727]
 [-0.06415518]
 [-0.3207759 ]
 [-0.0719371 ]
 [-0.01183789]
 [-0.05980805]]
[[ 6.57699024e+06 -2.58366281e+06 -2.60408337e+05  3.75964534e+05
   5.67989948e+05  2.54062713e+06 -5.15257467e+02  1.53183073e+02
  -6.70997008e+06  3.30490555e+04  1.59482930e+06 -6.47509450e+04
  -8.78522355e+05  7.05178871e+05 -1.58556464e+06  1.43109500e+03
  -1.88441169e+06]
 [-2.73811653e+06 -7.08015644e+14 -7.08015644e+13 -1.05224267e+14
  -1.57836400e+14 -6.42711786e+14  2.20811130e+02 -6.32436555e+01
   2.79242648e+06 -1.35482265e+04 -3.79651119e+14  2.70849009e+04
   2.68591009e+13  1.36231309e+14  3.79651119e+14 -5.74936598e+02
   5.26121333e+14]
 [-2.75855110e+05 -7.08015644e+13 -7.08015644e+12 -1.05224267e+13
  -1.57836400e+13 -6.42711786e+13  3.17476527e+01 -1.05175083e+01
   2.81499259e+05 -1.438909

In [95]:
th10

array([[-5.13486836],
       [-0.44142912],
       [-2.53126477],
       [-1.0320616 ],
       [-1.02832508],
       [-1.23737574],
       [ 0.18621226],
       [-0.13324414],
       [-0.33624383],
       [-1.28847161],
       [ 9.705266  ],
       [ 9.29640825],
       [-2.22432157],
       [ 0.44289757],
       [-5.53246552],
       [ 0.43992637],
       [ 3.44009835]])

In [96]:
c=0
for i in range(320):
    temp=te[i].reshape(1,17)
    if (hval(th10,temp)>=0.5 and teans[i]==1.0):
        c+=1
    elif (hval(th10,temp)<0.5 and teans[i]==0.0):
        c+=1
c

219